<a href="https://colab.research.google.com/github/Skeane-CD/Cargill_2025/blob/main/Cargill_LULC_Change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hey Shannon



You should be able to run this script to get the following outputs. First you will need to get the combine properties geojson from the cargill folder and the 'gfw_peat_clip_250815.tif' from the peat subfolder. Once downloaded, drag and drop them to the contents side bar. Remember: the contents are always temporary layers, be sure to download any outputs


1.) Each property is searched to see if it intersects with our peat data (currently the GFW data) This should be a binary layer

2.) The selected properties are then analyzed to measure their deforestation using the MapBiomas data

3.) The mapbiomas data is masked to our peat layer.

4.) the outputs include the unique 'code' name from the geospatial farm data, change as needed if this gets updated

5.) The csv table will include an initial and final forest cover, a start data and end data to the deforestation.

If you need to change the outputs, copy this script to ChatGPT, and ask it to help.


In [ ]:
# Run Me First!

!pip -q install geemap geopandas shapely rasterio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# navigate to your repo folder in Drive
# SHANNON -- WHEN THE POP-UP ASKS, RESTART THE SESSION, BUT DO NOT RERUN THIS CELL OR THE PRIOR CELLS
#            MOVE ONTO THE NEXT CELL AND RUN THAT ONE. YOU MAY NEED TO RUN THE NEXT CELL TWICE B/C IT WILL WANT TO
#            RE-AUTHENTICATE GDRIVE ---- BUT THIS SHOULD WORK @_@
%cd "/content/drive/MyDrive/Cargill/Colab/svante"
!pip install -e .

/content/drive/MyDrive/Cargill/Colab/svante
Obtaining file:///content/drive/MyDrive/Cargill/Colab/svante
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.7/912.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.1/454.1 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/9

In [ ]:
from svante.gcp import EarthEngineClient
EarthEngineClient()

In [ ]:
# --- Colab/EE setup ---
import ee, geemap, geopandas as gpd, pandas as pd, numpy as np, math
import rasterio
from rasterio.features import shapes as rio_shapes
from rasterio.mask import mask as rio_mask
from shapely.geometry import shape as shp_shape, mapping as shp_mapping
from shapely.ops import unary_union
from pathlib import Path


# Authenticate (first run in Colab will prompt you)
# try:
#    ee.Initialize(project = 'data-silo')
# except Exception:
#   ee.Authenticate()
#    ee.Initialize(project = 'data-silo')


In [ ]:

# ===================== USER PARAMETERS =====================

# 1) Local input: point to either a GeoJSON or a Shapefile (zip the shapefile or pass the .shp and sidecars exist)
LOCAL_VECTOR_PATH = "/content/cargill_properties_250722.geojson"  # e.g., "/content/aoi.geojson" or "/content/aoi.shp"


# 2) Column name in your vector that holds the unique identifier you want in outputs
CODE_FIELD = "code"   # <-- change if your file uses a different field name


# 3) Years in MapBiomas Indonesia Collection 2 (coverage)
START_YEAR = 2000
END_YEAR   = 2022

# 4) Forest class codes (adjust to your legend if needed)
# Code as needed: https://landy.mapbiomas.id/assets/legendcode/Indonesia%20Coll%202_Legend%20Code.pdf
FOREST_CLASS_IDS = [3]

# 5) Output options
OUT_CSV = "/content/deforestation_dynamics_c2.csv"


# 6) "Peat" binary raster input:
#    Provide either a GCS URL (gs://bucket/path.tif) or an EE asset ID (users/... or projects/...)
PEAT_LOCAL_PATH = "/content/gfw_peat_clip_250815.tif"  # or "users/you/peat_mask"
PEAT_VALID_VALUES = [1]                        # values treated as 'valid' in the mask



# Choose how results come back:
#   "paged" -> pull from EE in batches (stays in Colab; best for quick iteration)
#   "drive" -> server-side Export.table.toDrive (robust for very large results)
OUTPUT_MODE = "paged"   # "paged" or "drive"


# Drive export settings (only used if OUTPUT_MODE="drive")
DRIVE_FILENAME = "deforestation_dynamics_c2"
DRIVE_FOLDER   = None  # or "your-folder"


In [ ]:
# (If your mask has different valid codes, e.g. 1 and 255, set [1, 255])
# Tiling + simplification to keep requests tiny
PEAT_TILE_SIZE_PX = 1024        # tile width/height in pixels (e.g., 512–2048)
PEAT_SIMPLIFY_DEG = 0.0003      # simplify each tile's polygons (~30 m at equator). Increase if still heavy.
PEAT_DOWNSAMPLE_FACTOR = 1      # optional integer >1 to coarsen peat before polygonizing (e.g., 2, 4)

# Page size for OUTPUT_MODE="paged"
PAGE_SIZE = 1000


# Geom simplification (degrees). Increase if your polygons are extremely detailed.
# Vector simplification for AOIs (helps with payloads)
AOI_SIMPLIFY_DEG = 0.0005

# ---------------- Vector to EE ----------------
def local_vector_to_ee_fc(path: str, simplify_tol_deg: float, code_field: str) -> ee.FeatureCollection:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"Vector not found: {p}")

    gdf = gpd.read_file(str(p))
    if gdf.empty:
        raise ValueError("Input vector has no features.")
    if gdf.crs is None or gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(4326)

    if simplify_tol_deg and simplify_tol_deg > 0:
        gdf["geometry"] = gdf.geometry.simplify(simplify_tol_deg, preserve_topology=True)

    if "id" not in gdf.columns:
        gdf["id"] = [f"feat_{i}" for i in range(len(gdf))]

    if code_field not in gdf.columns:
        raise ValueError(f"'{code_field}' not found in columns: {list(gdf.columns)}")
    if code_field != "code":
        gdf["code"] = gdf[code_field]

    return geemap.geopandas_to_ee(gdf)

# ---------------- MapBiomas C2 stack ----------------
def load_mb_c2_year(y: int) -> ee.Image:
    url = f"gs://mapbiomas-public/initiatives/indonesia/collection_2/coverage/indonesia_coverage_{y}.tif"
    return ee.Image.loadGeoTIFF(url).rename(f"classification_{y}")

def build_mb_c2_stack(start_year: int, end_year: int) -> ee.Image:
    years = list(range(start_year, end_year + 1))
    return ee.Image.cat([load_mb_c2_year(y) for y in years])

def forest_mask(image_band: ee.Image, forest_ids: list) -> ee.Image:
    m = image_band.eq(forest_ids[0])
    for cid in forest_ids[1:]:
        m = m.Or(image_band.eq(cid))
    return m.rename("forest")

# ---------------- Local peat → tiled EE mask ----------------
def build_peat_mask_tiled(local_path: str,
                          valid_values: list,
                          aoi_gdf: gpd.GeoDataFrame,
                          tile_px: int,
                          simplify_deg: float,
                          downsample_factor: int = 1) -> ee.Image:
    """
    Crops peat to AOI, (optionally) downsamples, then iterates tiles:
      tile -> polygonize valid==1 -> dissolve -> simplify -> paint -> max accumulate
    Returns ee.Image('peat') with values {0,1}.
    """
    p = Path(local_path)
    if not p.exists():
        raise FileNotFoundError(f"Peat raster not found: {p}")

    # Read and crop to AOI in raster CRS to keep arrays as small as possible
    with rasterio.open(str(p)) as src:
        peat_crs = src.crs
        aoi_reproj = aoi_gdf.to_crs(peat_crs)
        aoi_union = unary_union(aoi_reproj.geometry.values)
        if aoi_union.is_empty:
            # No AOI; return empty mask
            return ee.Image(0).rename("peat").toByte()

        arr_crop, transform_crop = rio_mask(src, [shp_mapping(aoi_union)], crop=True)
        arr_crop = arr_crop[0]  # single band

    # Binarize (peat==1 where pixel in valid_values)
    valid = np.isin(arr_crop, np.array(valid_values)).astype(np.uint8)

    # Optional downsample to reduce complexity (max-pool)
    if downsample_factor and downsample_factor > 1:
        h, w = valid.shape
        new_h = math.ceil(h / downsample_factor)
        new_w = math.ceil(w / downsample_factor)
        pad_h = new_h * downsample_factor - h
        pad_w = new_w * downsample_factor - w
        valid_p = np.pad(valid, ((0, pad_h), (0, pad_w)), mode="constant", constant_values=0)
        valid_p = valid_p.reshape(new_h, downsample_factor, new_w, downsample_factor).max(axis=(1,3)).astype(np.uint8)

        # Update transform (pixel size increases by factor)
        a = transform_crop.a * downsample_factor
        e = transform_crop.e * downsample_factor  # note: typically negative
        c = transform_crop.c
        f = transform_crop.f
        transform_work = rasterio.Affine(a, 0, c, 0, e, f)
        valid = valid_p
    else:
        transform_work = transform_crop

    # Iterate tiles -> polygonize -> simplify -> paint
    H, W = valid.shape
    peat_mask_img = ee.Image(0).rename("peat").toByte()

    tiles_in_total = math.ceil(H / tile_px) * math.ceil(W / tile_px)
    printed = False

    for y0 in range(0, H, tile_px):
        for x0 in range(0, W, tile_px):
            y1 = min(H, y0 + tile_px)
            x1 = min(W, x0 + tile_px)
            tile = valid[y0:y1, x0:x1]
            if tile.sum() == 0:
                continue  # no peat in this tile

            # Transform for this tile
            # Xgeo = a*x + c ; Ygeo = e*y + f
            a, e, c, f = transform_work.a, transform_work.e, transform_work.c, transform_work.f
            tile_transform = rasterio.Affine(a, 0, c + x0 * a, 0, e, f + y0 * e)

            # Polygonize tile
            geom_iter = rio_shapes(tile, mask=tile == 1, transform=tile_transform)
            geoms = [shp_shape(geom) for geom, val in geom_iter if val == 1]
            if not geoms:
                continue

            # Dissolve to reduce feature count, project to WGS84, simplify
            dissolved = unary_union(geoms)
            gdf_tile = gpd.GeoDataFrame(geometry=[dissolved], crs=aoi_gdf.crs if aoi_gdf.crs else peat_crs)
            if gdf_tile.crs is None or gdf_tile.crs.to_epsg() != 4326:
                gdf_tile = gdf_tile.to_crs(4326)
            if simplify_deg and simplify_deg > 0:
                gdf_tile["geometry"] = gdf_tile.geometry.simplify(simplify_deg, preserve_topology=True)

            # Upload small vector and paint
            fc_tile = geemap.geopandas_to_ee(gdf_tile)
            tile_img = ee.Image(0).rename("peat").paint(fc_tile, 1).toByte()
            peat_mask_img = peat_mask_img.max(tile_img)  # union

            if not printed:
                print("Streaming peat tiles to EE… (this can take a bit, but stays under 10 MB per request)")
                printed = True

    return peat_mask_img

# ---------------- Metrics on mask ----------------
def compute_metrics_on_mask(fc: ee.FeatureCollection,
                            mb_img: ee.Image,
                            forest_ids: list,
                            peat_mask: ee.Image,
                            start_year: int,
                            end_year: int) -> ee.FeatureCollection:
    """Per-feature metrics restricted to peat==1, plus annual FA_YYYY (ha) columns."""
    area_image = ee.Image.pixelArea()  # m^2
    years = list(range(start_year, end_year + 1))

    # Peat area (ha) per feature for selection & transparency (unmasked so 0 comes back, not {})
    peat_area_ha_img = peat_mask.unmask(0).multiply(area_image).divide(1e4).rename("peat_area_ha")

    def add_peat_area(f):
        stats = peat_area_ha_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=f.geometry(), scale=30, maxPixels=1e13, bestEffort=True
        )
        peat_ha = ee.Number(ee.Algorithms.If(ee.Dictionary(stats).contains('peat_area_ha'),
                                             stats.get('peat_area_ha'), 0))
        return f.set({"peat_area_ha": peat_ha})

    fc_with_peat = fc.map(add_peat_area)
    fc_filtered  = fc_with_peat.filter(ee.Filter.gt("peat_area_ha", 0))

    # Per-year forest masks confined to peat
    forest_year = {
        y: forest_mask(mb_img.select(f"classification_{y}"), forest_ids).And(peat_mask).rename(f"forest_{y}")
        for y in years
    }

    # Start/end forest area (ha) inside peat
    start_forest = forest_year[start_year]
    end_forest   = forest_year[end_year]
    start_area_ha_img = start_forest.updateMask(start_forest).multiply(area_image).divide(1e4).rename("start_area_ha")
    end_area_ha_img   = end_forest.updateMask(end_forest).multiply(area_image).divide(1e4).rename("current_area_ha")

    # Annual forest area (ha) inside peat for all years
    fa_images = [
        forest_year[y].updateMask(forest_year[y]).multiply(area_image).divide(1e4).rename(f"FA_{y}")
        for y in years
    ]
    fa_stack = ee.Image.cat(fa_images)

    # Loss within peat: forest_{t-1} & !forest_{t}
    loss_year_imgs = []
    for y in range(start_year + 1, end_year + 1):
        prevF = forest_year[y - 1]
        currF = forest_year[y]
        loss  = prevF.And(currF.Not())
        loss_year_imgs.append(loss.multiply(y).rename(f"loss_{y}"))

    if loss_year_imgs:
        loss_stack = ee.ImageCollection(loss_year_imgs).toBands()
        first_loss = loss_stack.mask(loss_stack.gt(0)).reduce(ee.Reducer.min()).rename("first_loss_year")
        last_loss  = loss_stack.mask(loss_stack.gt(0)).reduce(ee.Reducer.max()).rename("last_loss_year")

        # MAIN metrics reducer (start/end + first/last loss)
        main_img = (start_area_ha_img
                    .addBands(end_area_ha_img)
                    .addBands(first_loss)
                    .addBands(last_loss))
        main_reducer = (
            ee.Reducer.sum().setOutputs(['start_area_ha_sum'])
            .combine(ee.Reducer.sum().setOutputs(['current_area_ha_sum']), sharedInputs=False)
            .combine(ee.Reducer.min().setOutputs(['first_loss_year_min']), sharedInputs=False)
            .combine(ee.Reducer.max().setOutputs(['last_loss_year_max']), sharedInputs=False)
        )

        def per_feat(f):
            # Reduce main metrics
            stats_main = main_img.reduceRegion(
                reducer=main_reducer, geometry=f.geometry(), scale=30,
                maxPixels=1e13, bestEffort=True
            )
            # Reduce FA bands separately (per-band sum; keys are band names)
            stats_fa = fa_stack.reduceRegion(
                reducer=ee.Reducer.sum(), geometry=f.geometry(), scale=30,
                maxPixels=1e13, bestEffort=True
            )
            # Compose output dict
            out = {
                "mask_name": "peat_local_tiled",
                "mask_source": PEAT_LOCAL_PATH,
                "start_year": start_year,
                "end_year": end_year,
                "start_forest_area_ha": ee.Number(stats_main.get("start_area_ha_sum")),
                "current_forest_area_ha": ee.Number(stats_main.get("current_area_ha_sum")),
                "first_loss_year": ee.Number(stats_main.get("first_loss_year_min")),
                "deforestation_stopped_year": ee.Number(stats_main.get("last_loss_year_max")),
            }
            out = ee.Dictionary(out).combine(ee.Dictionary(stats_fa), True)
            return f.set(out)

        return fc_filtered.map(per_feat)

    else:
        # Single-year fallback (still compute all FA_YYYY)
        main_img = start_area_ha_img.addBands(end_area_ha_img)
        main_reducer = (
            ee.Reducer.sum().setOutputs(['start_area_ha_sum'])
            .combine(ee.Reducer.sum().setOutputs(['current_area_ha_sum']), sharedInputs=False)
        )

        def per_feat_single(f):
            stats_main = main_img.reduceRegion(
                reducer=main_reducer, geometry=f.geometry(), scale=30,
                maxPixels=1e13, bestEffort=True
            )
            stats_fa = fa_stack.reduceRegion(
                reducer=ee.Reducer.sum(), geometry=f.geometry(), scale=30,
                maxPixels=1e13, bestEffort=True
            )
            out = {
                "mask_name": "peat_local_tiled",
                "mask_source": PEAT_LOCAL_PATH,
                "start_year": start_year,
                "end_year": end_year,
                "start_forest_area_ha": ee.Number(stats_main.get("start_area_ha_sum")),
                "current_forest_area_ha": ee.Number(stats_main.get("current_area_ha_sum")),
                "first_loss_year": None,
                "deforestation_stopped_year": None,
            }
            out = ee.Dictionary(out).combine(ee.Dictionary(stats_fa), True)
            return f.set(out)

        return fc_filtered.map(per_feat_single)

# ---------- Build dataset & run ----------
# Input polygons (carry 'code' into properties)
fc = local_vector_to_ee_fc(LOCAL_VECTOR_PATH, AOI_SIMPLIFY_DEG, CODE_FIELD)

# For cropping the peat locally we need the AOI as GeoDataFrame as well (for CRS/union)
gdf_aoi = gpd.read_file(LOCAL_VECTOR_PATH)
if gdf_aoi.crs is None or gdf_aoi.crs.to_epsg() != 4326:
    gdf_aoi = gdf_aoi.to_crs(4326)

# Local peat raster -> EE mask image via tiles
peat_mask = build_peat_mask_tiled(
    local_path=PEAT_LOCAL_PATH,
    valid_values=PEAT_VALID_VALUES,
    aoi_gdf=gdf_aoi,
    tile_px=PEAT_TILE_SIZE_PX,
    simplify_deg=PEAT_SIMPLIFY_DEG,
    downsample_factor=PEAT_DOWNSAMPLE_FACTOR
)

# MapBiomas C2 annual stack
mb_img = build_mb_c2_stack(START_YEAR, END_YEAR)

# Compute metrics ON peat only, and only for features overlapping peat
result_fc = compute_metrics_on_mask(
    fc=fc,
    mb_img=mb_img,
    forest_ids=FOREST_CLASS_IDS,
    peat_mask=peat_mask,
    start_year=START_YEAR,
    end_year=END_YEAR
)

# ---------- Output ----------
fa_cols = [f"FA_{y}" for y in range(START_YEAR, END_YEAR + 1)]
props = [
    "code","id","mask_name","mask_source","peat_area_ha",
    "start_year","end_year",
    "start_forest_area_ha","current_forest_area_ha",
    "first_loss_year","deforestation_stopped_year"
] + fa_cols

if OUTPUT_MODE == "drive":
    task = ee.batch.Export.table.toDrive(
        collection=result_fc.select(props),
        description="deforestation_dynamics_c2_on_peat",
        fileNamePrefix=DRIVE_FILENAME,
        folder=DRIVE_FOLDER,
        fileFormat="CSV",
    )
    task.start()
    print("Started Drive export task:", task.id)
else:
    size = result_fc.size().getInfo()
    print(f"Features to fetch (paged): {size}")

    rows, offset = [], 0
    while offset < size:
        batch_n = min(PAGE_SIZE, size - offset)
        page = ee.FeatureCollection(result_fc.select(props)).toList(batch_n, offset).getInfo()
        for feat in page:
            p = feat.get("properties", {})
            row = {
                "code": p.get("code"),
                "id": p.get("id"),
                "mask_name": p.get("mask_name"),
                "mask_source": p.get("mask_source"),
                "peat_area_ha": p.get("peat_area_ha"),
                "start_year": p.get("start_year"),
                "end_year": p.get("end_year"),
                "start_forest_area_ha": p.get("start_forest_area_ha"),
                "current_forest_area_ha": p.get("current_area_ha") if "current_area_ha" in p else p.get("current_forest_area_ha"),
                "first_loss_year": p.get("first_loss_year"),
                "deforestation_stopped_year": p.get("deforestation_stopped_year"),
            }
            for k in [f"FA_{y}" for y in range(START_YEAR, END_YEAR + 1)]:
                row[k] = p.get(k)
            rows.append(row)
        offset += batch_n
        print(f"Fetched {offset}/{size}")

    df = pd.DataFrame(rows)
    display(df)
    df.to_csv(OUT_CSV, index=False)
    print(f"Saved: {OUT_CSV}")

FileNotFoundError: Vector not found: /content/cargill_properties_250722.geojson